# Installation
Just pip install:

```pip install omegaconf```

If you want to try this notebook after checking out the repository be sure to run 
```python setup.py develop``` at the repository root before running this code.

# Creating OmegaConf objects
You can create an empty config:

In [ ]:
from omegaconf import OmegaConf
conf = OmegaConf.create()
print(conf)

Or a config from a dictionary:

In [ ]:
conf = OmegaConf.create(dict(k='v',list=[1,dict(a='1',b='2')]))
print(conf.pretty())

Or a config from a list:

In [ ]:
conf = OmegaConf.create([1, dict(a=10, b=dict(a=10))])
print(conf.pretty())

Or from from a yaml file:

In [ ]:
conf = OmegaConf.load('../source/example.yaml')
print(conf.pretty())

Or even a yaml string:

In [ ]:
yaml = """
a: b
b: c
list:
- item1
- item2
"""
conf = OmegaConf.create(yaml)
print(conf.pretty())

OmegaConf supports creating a configuration tree from a dot separated list:

This is typically used to override values using command line arguments. 
OmegaConf.from_cli() will parse the command line arguments in sys.argv.
You can also pass it a list for parsing.

In [ ]:
conf = OmegaConf.from_cli(['server.port=82', 'log.file=log2.txt'])
print(conf.pretty())

In [ ]:
%load '../source/example.yaml'

#### Object style access:

In [ ]:
conf = OmegaConf.load('../source/example.yaml')
conf.server.port

#### dictionary style access

In [ ]:
conf['log']['rotation']

#### items in list

In [ ]:
conf.users[0]

#### Changing existing keys

In [ ]:
conf.server.port = 81

#### Adding new keys

In [ ]:
conf.server.hostname = "localhost"

#### Adding a new dictionary

In [ ]:
conf.database = {'hostname': 'database01', 'port': 3306}

#### providing default values

In [ ]:
conf.missing_key or 'a default value'
conf.get('missing_key', 'a default value')

#### Accessing mandatory values
Accessing fields with the value *???* will cause a MissingMandatoryValue exception.
Use this to indicate that the value must be set before accessing.

In [ ]:
conf.log.file

# String interpolation
## Built-in resolvers
OmegaConf supports string interpolation. The basic form supports referencing other nodes in the configuration tree. Additionally, environment variable interpolation is supported.

Input yaml file:

In [ ]:
%load '../source/interpolation.yaml'

Interpolations are evaluated lazily on field access.<br/>
Note below that when printed the interpolations are not resolved.<br/>
They get resolved once you access them.

In [ ]:
conf = OmegaConf.load('../source/interpolation.yaml')
print(conf.pretty())

Primitive interpolations type is inherited from the referenced value.<br/>
Composite interpolations type is always string.


In [ ]:
def prn(s):
    print(f"{s} ({type(s).__name__})")
prn(conf.client.server_port)
prn(conf.client.url)

#### Environment variable can also be used

In [ ]:
print(conf.user.name)
print(conf.user.home)

## Plugable resolvers

You can add additional interpolation types using custom resolvers. This example creates a resolver that adds 10 the the given value.

In [ ]:
OmegaConf.register_resolver("plus_10", lambda x: int(x) + 10)
conf = OmegaConf.create({'key': '${plus_10:990}'})
print(conf.key)

# Merging configurations
Merging configurations enables the use of logical components instead of a single config file for each variation of your task.

Machine learning experiment example:
```python
conf = OmegaConf.merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

Web server configuration example:

```python
conf = OmegaConf.merge(server_cfg, plugin1_cfg, site1_cfg, site2_cfg)
```

The following example creates two configs from files, and one from the cli. it then combines them into a single object. Note how the port changes to 82, and how the users list is combined.

In [ ]:
%load '../source/example2.yaml'

In [ ]:
%load '../source/more_users.yaml'

In [ ]:
from omegaconf import OmegaConf
import sys
base_conf = OmegaConf.load('../source/example2.yaml')
users_conf = OmegaConf.load('../source/more_users.yaml')
# Simulate command line arguments
sys.argv = ['program.py', 'server.port=82']
cli_conf = OmegaConf.from_cli()
# Overlay configs:
conf = OmegaConf.merge(base_conf, users_conf, cli_conf)
print(conf.pretty())